#Faster Whisper 語音辨識測試

update: 2023.10.23 by gsyan
* 可使用 .txt 檔案放要辨識的清單。
* 可自訂字幕檔要儲存的資料夾。
* 可自訂是否要跳過已辨識過的。
* 使用 yt-dlp 下載影音檔案。

本工具使用的是 Faster Whisper ，它的專案網址如下：
* https://github.com/guillaumekln/faster-whisper

從專案的名稱「Faster Whisper」就可以看出它強調的特性，據專案的說明，它提到比 OpenAI Whisper 的速度快。以 2023.10.23 實測來比較，至少 faster-whisper 的安裝就可以少很多時間。

在本筆記中的這個工具結合 Faster Whisper 和 yt-dlp 的工具，可以將 Youtube 上的影片或播放清單擷取聲音、儲存語音檔後，進行語音辨識，再利用 pysubs2 把辨識完的文字轉成有加時間戳記的字幕檔，當然也可以是沒有時間戳記的逐字稿文字檔。

目前在後面程式設定區塊中，語音來源路徑的「url」欄位中，可以填入：
* Youtube 的影片或影片清單網址。
* 公開分享的 Google 雲端硬碟、FB ......影音網址。
* Vocaroo 的聲音網址。如果想馬上錄一段做測試，也可以利用線上錄音的網站 [Vocaroo](https://vocaroo.com/) (https://vocaroo.com/) 來錄音，再將它給的網址貼在「url」欄位中；
* 在 Colab 中的檔案路徑。如果是放在電腦本機中的影片或聲音檔，可以上載在左側資料夾中後，在「url」欄位中填入完整的檔案名稱。
* 「檔案清單」文字檔(.txt)。如果想一次辨識多個檔案，就將網址或是檔案的路徑，一行一個檔案路徑置入文字檔中，將文字上載後，將清單檔案的路徑填入 [url] 欄位中。

接著將其它選項都設定好後，就可以在[程式區塊]中按「執行」的按鈕，開始進行語音辨識了。

程式第一次執行時，因為要安裝及下載自動語音辨識所需要的資料，可能要稍等一下下。


關於 OpenAI Whisper 的專案，可以參考我 Blogger 中的說明，網址如下：
* https://gsyan888.blogspot.com/2023/02/openai-whisper-ipynb.html




In [ ]:
#@title Faster Whisper 語音辨識並輸出字幕檔案或是一般文字檔案程式 { vertical-output: true }

#@markdown <b>設定底下的自訂參數後，就可以按左側的執行鈕</b>

# @markdown <b>聲音檔的來源</b>，可以是網址(Youtube影片、撥放清單、 [Vocaroo](https://vocaroo.com/) 網址)，或是上載後的影片、聲音檔檔案名稱。<br />有多個可以放入 .txt 檔案中(例: list.txt)，一行一個網址或是檔案路徑。
url = "https://www.youtube.com/watch?v=84AalWF4qUg" #@param {type:"string"}
#@markdown <b>語音的語言代碼</b>(兩個小寫字母, zh 中文、en 英語、ja 日語...)
#lang = 'Chinese' #@param ["Chinese", "English", "Japanese", "Korean", "自動判斷"]
lang = "zh" #@param ["自動判斷", "en", "zh", "ja", "fr", "de"] {allow-input: true}

#@markdown <b>輸出為哪一種格式</b>（.srt:字幕檔、.txt:純文字檔）
outputFormat = 'srt' #@param ['srt', 'txt']
#@markdown <b>輸出到哪個資料夾</b>(空白表示使用預設值)
output_path = '' #@param {type:"string"}
#@markdown <b>是否覆蓋已存在的辨識結果</b>(沒勾選會跳過已辨識過的)
overwrite_enable = True #@param { type: 'boolean' }
# @markdown <b>使用哪一種辨識模型</b>（small:快，medium,large-v2:精準）
modelType = "large-v2" # @param ["tiny", "tiny.en", "base", "base.en", "small", "small.en", "medium", "medium.en", "large-v1", "large-v2", "large"] {allow-input: true}
#@markdown <b>是否全部辨識完成，立即下載字幕檔</b>
start_downloading_immediately = True #@param { type: 'boolean' }
#@markdown <b>是否即時顯示語音辨識結果</b>
verbose = True #@param { type: 'boolean' }
#markdown <b>是否使用 yt-dlp 下載 Youtube 聲音檔</b><br />(如果較長的聲音檔下載發生問題時再勾選使用)
#YoutubeDL_enable = False #@param { type: 'boolean' }

#@markdown ---
#@markdown <b>其它進階選項 (使用預設值即可，有特別需要再更改)</b>
compute_type = "float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ##### Word-level timestamps(以字為單位切時間)
word_level_timestamps = False #@param {type:"boolean"}
#@markdown ##### VAD filter
vad_filter = True #@param {type:"boolean"}
vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
#@markdown <b>提示語</b> (空白即可，如果想輸出繁體中文, 卻變簡體字再使用)
initial_prompt = "" # @param ["", "\u4EE5\u4E0B\u662F\u7E41\u9AD4\u4E2D\u6587\u7684\u53E5\u5B50", "\u4EE5\u4E0B\u662F\u666E\u901A\u8BDD\u7684\u53E5\u5B50\u3002"] {allow-input: true}

#@markdown ---
#@markdown <center><p><font color="blue">以下為指令輸出內容</p></center>




# Install + Import + Config
#try: import whisper
#except:
#  print('install whisper ...')
#  ! pip -q install git+https://github.com/openai/whisper.git

#huggingface_hub 0.20 以後必須設定 HF_TOKEN, 所以先安裝舊版的覆蓋避開
print('install old version huggingface_hub==0.19.4 ...')
!pip -q install huggingface_hub==0.19.4

try:
  from faster_whisper import WhisperModel
except:
  print('install faster-whisper ...')
  !pip -q install faster-whisper

try:
  import pysubs2
except:
  print('install pysubs2 ...')
  !pip -q install pysubs2

try: from yt_dlp import YoutubeDL
except :
  print('install yt_dlp ...')
  ! pip -q install yt_dlp


try: from slugify import slugify
except:
  print('install python-slugify ...')
  ! pip install python-slugify


import torch
#import whisper
#from whisper.utils import get_writer

from faster_whisper import WhisperModel
import pysubs2

from yt_dlp import YoutubeDL
import re
import os
import urllib.request
from slugify import slugify

import google
from IPython.display import display, Markdown, YouTubeVideo

#import yt_dlp

#url = "https://voca.ro/15HVH0YvIaa6"
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ&list=PLelNvYGEtsV8TpwxL4t7GTTG-7qALZqol'
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ'
#url = 'vocaroo-台語.mp3'

#lang = 'Chinese' # '自動判斷'
#start_downloading_immediately = True

#modelType = 'small' # 'small' 'medium'
#outputFormat = 'txt' # 'srt' 'txt'

audioFile = 'source.mp3' #暫存的語音檔檔名
tempFolder = '.' #暫存的資料夾(工作目錄、下載的影音、剛轉好的文字檔)
#output_path = '.'
title = ''
textFileList = [] #記錄轉好的文字檔檔名清單

#檢查輸出文字檔的資料夾是否存在, 不存在就改用預設值
if re.sub('\s', '', output_path) == '' :
  #如果沒有輸入，就輸出到預設的暫存資料夾
  output_path = tempFolder
else :
  if os.path.exists(output_path) :
    if not os.path.isdir(output_path) :
      print(output_path + ' 不是資料夾哦! 改用預設值: '+tempFolder)
      output_path = tempFolder
  else :
    print('指定的輸出目錄 '+output_path+' 不存在, 改用預設值: '+tempFolder)
    output_path = tempFolder


# GPU or CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def getYoutubePlaylistInfo_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 播放清單網址擷取各影片的資訊
  :param url: Youtube 影片播放清單網址
  """
  ydl_opts = {
      'no_warnings':True,
      'quiet':True,
      #'extract_flat':True
  }
  with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    #return (info_dict.get('entries', None)) #entries 為清單中的影片, 'webpage_url' 為影片網址
    return info_dict

def getYoutubeTitle_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 網址擷取影片的標題字
  :param url: Youtube 影片網址
  """
  ydl_opts = {
      'no_warnings':True,
      'quiet':True
  }
  with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    return(info_dict.get('title', None))

def getAudioFromYoutube_ydl(url, output_path, outputFilename):
  """
  使用 yt_dlp 由 Youtbue 網址下載影片的聲音並存檔
  :param url: Youtube 影片網址
  :param output_path: 儲存的資料夾名稱
  :param outputFilename: 聲音檔的檔名
  """
  #filename為去掉副檔名的路徑，以免變成 xxx.mp3.mp3
  filename = os.path.join(output_path, os.path.splitext(outputFilename)[0])
  ydl_opts = {
      'no_warnings':True,
      'quiet':True,
      'outtmpl':filename,
      'keepvideo':False,
      'format': 'bestaudio/best',
      'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
      #  'preferredquality': '192',
      }],
      'overwrites':True
  }
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(url)

def getVocarooMP3URL(url) :
  """
  Get the MP3 URL from Vocaroo record share URL
  :param url: Vocaroo url
  """
  vocarooMP3Base = 'https://media.vocaroo.com/mp3/'
  regex = re.compile(r'(https\:\/\/voca\.ro|https\:\/\/vocaroo\.com)\/(\w{12})')
  match = regex.search(url)
  if match :
    url = vocarooMP3Base+match.group(2)
  return url

def transcribe(audioFilename, output_path, outputFilename, outputFormat) :
  """
  load whisper model and get transcribe result
  :param audioFilename: audio file to transcribe
  :param output_path: the folder to save result
  :param outputFilename: the filename(without the extension) to save result
  :param outputFormat: the transcribe result format (thie extension of filename)
  """
  #print(audioFilename+' --> '+output_path+outputFilename+'.'+outputFormat)
  #return 1

  #model = whisper.load_model(modelType, device=DEVICE)
  model = WhisperModel(modelType, device=DEVICE, compute_type=compute_type)

  segments, info = model.transcribe(audioFilename, beam_size=5,
                                  language=None if lang == "自動判斷" else lang,
                                  initial_prompt=None if initial_prompt == "" else initial_prompt,
                                  word_timestamps=word_level_timestamps,
                                  vad_filter=vad_filter,
                                  vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

  #display(Markdown(f"偵測到的語言 '{info.language}' 可能性 {info.language_probability}"))
  display(Markdown("偵測到語言: '%s' , 可能性: %s%%" % (info.language, info.language_probability*100)))
  file_name = getOutputTextFilename(outputFilename)
  if outputFormat=='txt' :
    with open(file_name, 'w') as f:
      for segment in segments:
        if word_level_timestamps:
          for word in segment.words:
            if verbose :
              print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
            f.write(f"{word.word}")
          f.write("\n")
        else:
          if verbose :
            print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
          f.write(f"{segment.text.strip()}\n")
  else:
    results= []
    for segment in segments:
      if word_level_timestamps:
        for word in segment.words:
          if verbose :
            print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
          # list for pysubs2
          segment_dict = {'start':word.start,'end':word.end,'text':word.word}
          results.append(segment_dict)
      else:
        if verbose :
          print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
        # list for pysubs2
        segment_dict = {'start':segment.start,'end':segment.end,'text':segment.text}
        results.append(segment_dict)
    subs = pysubs2.load_from_whisper(results)
    #save srt file
    subs.save(file_name)
    #save ass file
    #subs.save(file_name+'.ass')

  #if lang=="自動判斷" :
  #  print('auto detect language')
  #  result = model.transcribe(audioFilename, fp16=False, verbose=verbose)
  #else :
  #  result = model.transcribe(audioFilename, fp16=False, verbose=verbose, language=lang)
  # save result to outputFormat file
  #saveToFile(result, output_path, outputFilename, outputFormat)

def saveToFile(result, output_path, filename, fileType='srt') :
  """
  save whisper transcribe result to a file
  :param result: whisper transcribe result
  :param output_path: the folder to save result
  :param filename: the filename(without the extension) to save result
  :param fileType: the transcribe result format (thie extension of filename)
  """
  # save SRT(full filename : output_path/filename.fileType)
  file_writer = get_writer(fileType, output_path) #whisper get_writer
  #file_writer(result, filename)
  #2023.04.12 因應可標定字的時間截記 word_timestamps 必須傳入 writer_args 參數
  writer_args = {'highlight_words': False, 'max_line_count': None, 'max_line_width': None}
  file_writer(result, filename, writer_args)


#
def getFilenameFromTitle(title) :
  """
  convert title to valid filename
  :param title: text to slugify
  """
  return slugify(title, allow_unicode=True, lowercase=False)

def getOutputTextFilename(filename) :
  fPath = os.path.relpath(output_path)
  return os.path.join(fPath, filename+'.'+outputFormat)

def isOutputTextFileExists(filename) :
  return os.path.exists(getOutputTextFilename(filename))

def removeAudioFile(filename) :
  if os.path.exists(filename) :
    os.remove(filename)

def convertNewLineFormat(filename) :
  if os.path.exists(filename) :
    f = open(filename, 'r+')
    data = f.read()
    data = re.sub('\r', '', data, flags=re.MULTILINE) #remove all \r
    data = re.sub('\n', '\r\n', data, flags=re.MULTILINE) #replace \n to \r\n
    f.seek(0)
    f.write(data)
    f.truncate()
    f.close()

def getAudioAndTranscribe(urlOrFile) :
  filename = audioFile #語音檔檔名(辨識用的暫存檔，用固定的名稱較方便)
  # 分網址或是Colab儲存空間檔案處理
  if re.search('https\:\/\/', urlOrFile) :
    isPlayList = False
    # Youtube 分播放清單跟單一影片處理
    if re.search('youtube\.|youtu\.', urlOrFile) :
      # 建立 Playlist 物件
      #pList = Playlist(urlOrFile)
      videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
      isPlayList = True
      #try : title = pList.title
      try : videos = videoInfo['entries']
      except :
        isPlayList = False
      #處理 Youtube 播放清單
      if isPlayList :
        title = videoInfo['title']
        print('\n>>處理 Youtube 播放清單影片: '+title)
        #for video in pList.videos :
        for video in videos :
          #title = video.title #取得Youtube影片的標題字
          title = video['title'] #取得Youtube影片的標題字
          #filename = title+'.mp3'
          print('\n>>> '+title)
          #用影片的標題當主檔名，但需將不適合的字置換或去除
          #convert title to valid filename
          outputFilename = getFilenameFromTitle(title)
          #continue

          #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
          if not isOutputTextFileExists(outputFilename) or overwrite_enable :
            # delete the old audio file
            removeAudioFile(filename)
            # download audio from video stream
            #video.streams.get_audio_only().download(tempFolder, filename)
            print('>> 下載影音檔案中...')
            try: getAudioFromYoutube_ydl(video['webpage_url'], tempFolder, filename)
            except:
              print('>>>無法由網址下載影音')

            #print('\n>> 準備進行語音辨識...\n')
            if os.path.exists(filename) :
              print('>>>Whisper 辨識中...\n')
              # load whisper model and get transcribe result
              transcribe(filename, output_path, outputFilename, outputFormat)
            else :
              print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')

          #將輸出的檔名列入結果清單中，方便後續壓縮、下載
          textFileList.append(getOutputTextFilename(outputFilename))
        #title = pList.title #供製作下載壓縮檔用的主檔名
        title = videoInfo['title']
      else :
        # Youtube single video
        #title = YouTube(urlOrFile).title #取得Youtube影片的標題字
        title = videoInfo['title']
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          # delete the old audio file
          removeAudioFile(filename)
          #getAudioFromYoutube(urlOrFile, tempFolder, filename)
          print('>> 下載影音檔案中...')
          try: getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
          except:
            print('>>>無法由網址下載影音')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
    else :
      #非Youtube的網址
      # delete the old audio file
      removeAudioFile(filename)
      isPlayList = False
      if re.search('drive\.google\.com', urlOrFile) :
        #Google drive files
        title = getYoutubeTitle_ydl(urlOrFile) #取得檔名
        outputFilename = getFilenameFromTitle(title) #主檔名，但需將不適合的字置換或去除
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          getAudioFromYoutube_ydl(urlOrFile, tempFolder, filename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
      elif re.search('voca\.ro|vocaroo\.com', urlOrFile) :
        # Vocaroo or other web audio
        urlOrFile = getVocarooMP3URL(urlOrFile) #取得儲存 Vocaroo 的語音檔網址
        title = "vocaroo-"+os.path.basename(urlOrFile) #用 Vocaroo 的 id 當標題字
        outputFilename = title #用 vocaroo-xxxxxxxxxxx 當輸出的主檔名
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          urllib.request.urlretrieve(urlOrFile, filename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
      else :
        # other website
        #os.remove(filename)
        title = ''
        removeAudioFile(filename)
        print('\n>> 下載影音檔案中...\n')
        try : videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
        except:
          print('\n>>> 無法下載該網址的影音資訊 ...\n')
        try: title = videoInfo['title'] #取得標題字
        except:
          title = 'unknow'
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        try:
          getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
        except:
          print('\n>>> 無法下載影音\n')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
        #! yt-dlp -q --force-overwrites -x --audio-format mp3 -o {audioFile} {url}
  else :
    #Colab儲存空間檔案處理(非網址的影音檔)
    isPlayList = False
    title = os.path.splitext(os.path.basename(urlOrFile))[0] #去掉資料夾名稱及附檔名，只取主檔名
    outputFilename = getFilenameFromTitle(title) #結果輸出檔的主檔名
    filename = urlOrFile #辨識指定的影音檔
    if os.path.exists(filename) :
      #將輸出的檔名列入結果清單中，方便後續壓縮、下載
      textFileList.append(getOutputTextFilename(outputFilename))
      #如果字幕檔已存在，而且不可覆蓋，就故意將語音檔設為不存在的檔名，這樣就不會重新辨識
      if (not overwrite_enable) and isOutputTextFileExists(outputFilename) :
        filename = 'fakeFilename.fake'
  #辨識非播放清單的單一語音檔
  if not isPlayList :
    print('\n>> '+title)
    #convert title to valid filename
    #outputFilename = slugify(title, allow_unicode=True, lowercase=False)
    #outputFilename = getFilenameFromTitle(title)
    # 語音檔存在的話，就進行辨識
    # load whisper model and get transcribe result
    print('\n>> 準備進行語音辨識...\n')
    if os.path.exists(filename) :
      if not os.path.exists('/root/.cache/huggingface/hub'):
        print('>>> 請稍候, 下載完 Whisper 的 model 檔案後，即可開始辨識...\n')
      else :
        print('>>> 開始辨識...\n')
      transcribe(filename, output_path, outputFilename, outputFormat)
    else :
      print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')
    #  start_downloading_immediately = False
  return title

#解析 .txt 檔案中的清單，一行行進行去抓音檔並進行辨識
def parseTxtFileAndTranscribe(txtFile) :
  if os.path.exists(txtFile) :
    file = open(txtFile, 'r')
    lines = file.readlines()
    file.close()
    #print(len(lines))
    for line in lines :
      line = re.sub('\r|\n', '', line, re.MULTILINE) #去掉所有換行字元
      getAudioAndTranscribe(line)
  else :
    print('找不到檔案: '+txtFile)

if (re.search('https\:\/\/', url) is None) and (not re.search('\.txt$', url, re.IGNORECASE) is None) :
  #解析 .txt 檔中的清單後，再進行辨識
  parseTxtFileAndTranscribe(url)
  title = os.path.splitext(os.path.basename(url))[0] #去掉資料夾名稱及附檔名，只取主檔名
else :
  #直接辨識 url 指定的網址或是檔案
  title = getAudioAndTranscribe(url)


#print(textFileList)
if len(textFileList)>0 and start_downloading_immediately :
  print('download...')
  #print(textFileList)
  filenames_list = ''
  for textFile in textFileList :
    #print(textFile)
    if os.path.exists(textFile) :
      convertNewLineFormat(textFile) # replace \n with \r\n (Windows format)
      if os.getcwd() != os.path.dirname(os.path.abspath(textFile)) :
        filename = os.path.basename(textFile)
        #copy textFile to current work directory
        ! cp {textFile} ./
      else :
        filename = textFile
      if filenames_list != '' :
        filenames_list = filenames_list+' '
      filenames_list = filenames_list+filename
  #print(filenames_list)

  if len(textFileList)>1 :
    print('\n壓縮並下載辨識結果')
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
    ! zip {outputFilename} -D {filenames_list}
  else :
    print('\n下載辨識結果')
    outputFilename = filenames_list

  google.colab.files.download(outputFilename)

  '''
  if isPlayList :
    print('\n壓縮並下載辨識結果')
    #title = pList.title
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
    #先將所有字幕檔的換行符號由 \n 換成 \r\n，全部壓縮
    ! zip -l {outputFilename} *.{outputFormat}
  else :
    print('\n下載辨識結果')
    outputFilename = outputFilename+'.'+outputFormat
  #下載辨識結果
  google.colab.files.download(outputFilename)
  '''

install faster-whisper ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 122.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:00
install pysubs2 ...
install yt_dlp ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

偵測到語言: 'zh' , 可能性: 100%

[0.94s -> 5.12s] HTML5 好康的大小寶,好康的來喔!
[5.12s -> 7.90s] 筆順練習新增了一個
[7.90s -> 12.30s] 可以匯出練習筆跡為圖片的功能
[12.30s -> 15.99s] 我們看一下這個新的功能
[15.99s -> 20.07s] 這個匯出它本來是只能匯出設定檔
[20.07s -> 22.07s] 然後再利用這個匯入
[22.07s -> 28.31s] 來將練習的結果變成動畫
[28.31s -> 30.31s] 那我們按一下這個按鈕
[31.59s -> 33.59s] 可以看到這裡變成三個按鈕
[33.59s -> 37.86s] 匯出、洗寫、記錄檔就是原來的功能
[37.86s -> 39.86s] 那我們今天要來測試一下
[39.86s -> 43.13s] 下載筆跡為圖片這個功能
[43.13s -> 47.73s] 這邊畫面上有徽章的都代表已經寫過的字
[47.73s -> 51.54s] 沒有徽章的代表是還沒練習過的
[51.54s -> 53.94s] 我們按一下下載筆跡為圖片
[53.94s -> 59.47s] 按下去以後這裡就會下載了一個圖片檔
[59.47s -> 63.10s] 我們打開來看看它樣子
[63.10s -> 66.62s] 可以看到這個有手寫的筆跡
[66.70s -> 68.70s] 其他是沒有的
[68.70s -> 73.94s] 另外在左上角加上了匯出的時間
[73.94s -> 76.29s] 這個是新的功能
[76.29s -> 80.77s] 我們可以看到這邊有的是黑色的
[80.77s -> 83.63s] 然後有的是紅色的
[83.63s -> 86.63s] 黑色的部分就是正確的部分
[86.63s -> 90.63s] 紅色的就代表它寫錯的
[90.63s -> 95.80s] 我們來做一下實驗
[95.80s -> 99.11s] 比如說寫出這個字的時候
[99.11s -> 101.27s] 這裡是正確的
[101.55s -> 109.26s] 然後寫到這邊的時候故意寫錯
[109.26s -> 112.38s] 隨便亂寫
[112.38s -> 114.38s] 我們等一下看看這個地方
[114.38s -> 118.42s] 這個

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>